# Lecture on the Wigner's semicircle law

    by M.Süzen
    (c) 2025

In this lecture, we will generate the spectral density $ \rho(\lambda)$ emprically from a sample of matrices sampled from the Gaussian Orthogonal Ensemble (GOE) and compare against the exact solution for it's distribution using the `leymosun` package.

## Load Leymosun components

We assume that you have installed the package. We will import the following:

`goe` : Gaussian Orthogonal Matrix generator.   
`ensemble`: Ensemble generator.   
`offdiagonal`: Extract offdiagonal elements from a square matrix.   
`pdf`: This computes density for given values and equidistant location centres.  
`empirical_spectral_density`: Compute eigenvalues and their density $\rho(\lambda)$.     
`wigner`: Compute the Wigner density at a given $\lambda$.

See each method's documentation for more details. 

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import leymosun
from leymosun.gaussian import goe, wigner
from leymosun.matrix import ensemble, offdiagonal
from leymosun.spectral import pdf, empirical_spectral_density
from leymosun.stats import bootstrap_observed_matrix_ci

leymosun.__version__

## Gaussian Orthogonal Ensemble (GOE)

Gaussian ensemble starts with generating gaussian random numbers with the following form, so-called $\beta$-ensembles.

$G_{\beta=1} \sim \mathcal{N}(0, \sigma)$.

Here, the understanding is as follows: at $\beta=1$ ensemble, essentially we draw $N x N$ numbers forming a square matrix $G_{1}$. The following matrix $A^{GOE}(n)$ would be a sample from GOE ensemble:

$A^{GOE}(n) = \frac{1}{2}(G_{1} + G_{1}^{T})$.

$T$ is the transpose matrix. This matrix will have the following properties.

1. Diagonals with the following distribution: $diag(A^{GOE}) \sim \mathcal{N}(0, \sigma)$.
2. Offdiagonals with the foloowing distributions: $offdiag(A^{GOE}) \sim \mathcal{N}(0,\sigma/2)$. 

**Exercise 1** How come diagonal have a different variance?

Let's try to generate 100x100 matrix as an example and verify its properties with `leymosun`, where $\sigma=1.0$ is the default value.

In [ ]:
N = 10000
A = goe(N)
A_diag = np.diag(A)
A_offdiag = offdiagonal(A)
f"Diagonal Mean={float(np.mean(A_diag)):.2f}", \
f"         Variance={float(np.var(A_diag)):.2f}",  \
f"Offdiagonal Mean={float(np.mean(A_offdiag)):.2f}", \
f"           Variance={float(np.var(A_offdiag)):.2f}"


We see that variances appraoch to $1.0$ and $0.5$ as we increase the matrix size. Now we can visualise their densities, as follows.

In [ ]:
locations = np.arange(-5.0, 5.2, 0.2) 
density_diag, _locations = pdf(A_diag, locations)
density_offdiag, _locations = pdf(A_offdiag, locations)
plt.plot(_locations, density_diag, label="Diagonal distribution")
plt.plot(_locations, density_offdiag, label="Offdiagonal distribution")
plt.xlabel("GOE diagonal element values")
plt.ylabel("GOE diagonal element density")
plt.legend()

## Semicircle law

The semicircle law traces its origins to Wigner's work on the calculations of the quantum mechanical Hamiltonian [Wigner55]. The resulting distribution of the eigenvalues follows the so-called semicircle law. The probability density reads as follows:
$$ \rho(\lambda) = \frac{2}{\pi \cdot R^{2}} \sqrt{R^{2}-\lambda^{2}}$$
where by $\lambda$ is the eigenvalue. This density is a semicircle in between $[-R, R]$, R is $2 \cdot \sigma$. 


## Numerical Experiment

We conduct out numerical experiment using the tools provided by `leymosun` package. Emprical GOE ensemble of size 100 is generated for square matrices of 1000 by 1000. Then we compute the empirical spectral density in between $[-2,2]$. 

In [ ]:
matrix_order = 1000
ensemble_size = 100
ensemble_sample = ensemble(matrix_order, ensemble_size, goe)

In [ ]:
eigenvalues, densities, locations = empirical_spectral_density(
    ensemble_sample, scale="wigner",  locations = np.arange(-2.05, 2.1, 0.05),

)

We will compute 95% confidence intervals. 

In [ ]:
observed_mean, observed_upper, observed_lower = bootstrap_observed_matrix_ci(densities)

In [ ]:
dwigner = wigner(locations, domasigmin_boundary=2.0)

## Visualisation

We visalise the results in a single graph. A continous red line is the exact Wigner density from the theory, and bars are from our numerical experiments. 

In [ ]:
yerr = np.array([observed_mean-observed_lower, observed_upper-observed_mean])
plt.bar(locations, observed_mean, width=0.02, alpha=0.3, align='center') 
plt.errorbar(locations, observed_mean, yerr=yerr, fmt=' ', capsize=5)  
plt.plot(locations, dwigner, color='red')
plt.title("Wigner's Semicircle law with Leymosun")
plt.xlabel("Eigenvalue Locations")
plt.ylabel("Density")

It is important to note that Wigner's domain boundary and emprical spectral density should be the same, so we match the numerical experiments with the theory.

## Conclusion

We observe good match for demonstration purposes here. One can study how close they get with different generated ensemble size and matrix orders.  

**Exercise 2** Conduct numerical experiments to demonstrate semicircle-law as in above, but using the domain [-3.5, -3.5].  
**Hint** Use `partial` function for `goe` to set the `scale` ($\sigma$) accordingly.

## Reference

[Wigner55] Characteristic Vectors of Bordered Matrices With Infinite Dimensions, Eugene P. Wigner, Annals of Mathematics, Vol. 62, No. 3 (Nov., 1955), pp. 548-564

QED